## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

### LDA on a real world dataset ###

We will be using the 20 Newsgroup dataset which is a collection of emails exchanged in 20 newsgroups. You can read more about the dataset [here](http://qwone.com/~jason/20Newsgroups/).

## Step 1: Load the dataset ##

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics. Some of these topics are similar, so we will try to classify them into 10 topics.

Remove all email headers, footers and quotes from the replied emails.

In [32]:
'''
Dataset from http://scikit-learn.org/stable/datasets/twenty_newsgroups.html
'''
import numpy as np
np.random.seed(400)
from sklearn.datasets import fetch_20newsgroups
documents = fetch_20newsgroups(remove=('headers', 'footers', 'quotes')).data

In [33]:
'''
Get the total number of documents
'''
print(len(documents))

11314


In [66]:
# Previewing a document
document_num = 5288
print("\n**Printing out a sample document:**")
print(documents[document_num])


**Printing out a sample document:**
Have anybody succeded in converting a atari monomchrome monitor into a
mono VGA monitor. If so please let me know exactly how you did and what
graphics card you used.

	/Thanx


** Looks like this document will come under `comp.graphics`. **

## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [67]:
'''
Loading Gensim library
'''
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [38]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /Users/mat/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
'''
Lemmatizing example for a verb, noun.
'''
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


In [41]:
'''
Stemming example
'''
stemmer = SnowballStemmer("english")
plurals = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in plurals]
print(' '.join(singles))

caress fli die mule deni die agre own humbl size meet state siez item sensat tradit refer colon plot


In [42]:
'''
Data preprocessing
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result = [lemmatize_stemming(token) for token in gensim.utils.simple_preprocess(text) 
              # Remove stop words and words less than 3 characters long
              if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3]
    return result

In [43]:
'''
Preview a document after preprocessing
'''
print("Original document: ")
words = []
for word in documents[document_num].split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(documents[document_num]))

Original document: 
[':As', 'a', 'matter', 'of', 'fact,', 'i', 'do', 'keep', 'random', 'files', 'on', 'my', 'disk.', '', 'The', 'reason', 'is,\n:without', 'special-purpose', 'hardware,', 'it', 'takes', 'a', 'long', 'time', 'to', 'generate', 'good', 'random\n:bits.', '', 'I', 'have', 'programs', 'that', 'crank', 'out', 'a', 'couple', 'bits', 'per', 'minute,', 'which', 'is\n:pretty', 'conservative,', 'but', 'over', 'time', "that's", 'more', 'than', 'i', 'need.\n\nSounds', 'like', 'a', 'useful', 'program', '-', 'interested', 'in', 'posting', 'it', 'to', 'alt.sources?']


Tokenized and lemmatized document: 
['matter', 'fact', 'random', 'file', 'disk', 'reason', 'special', 'purpos', 'hardwar', 'take', 'long', 'time', 'generat', 'good', 'random', 'bit', 'program', 'crank', 'coupl', 'bit', 'minut', 'pretti', 'conserv', 'time', 'need', 'sound', 'like', 'use', 'program', 'interest', 'post', 'sourc']


In [44]:
'''
Perform preprocessing on entire dataset
'''
processed_docs = [preprocess(doc) for doc in documents]

## Step 3.1: Bag of words on the dataset ##

In [45]:
'''
Create a dictionary containing the number of times a word appears in the training set
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [46]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 addit
1 bodi
2 bricklin
3 bumper
4 call
5 door
6 earli
7 engin
8 enlighten
9 funki
10 histori


** Gensim filter_extremes **

`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [47]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, 
                                no_above=0.10)

** Gensim doc2bow **

`doc2bow(document)`

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [48]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear.
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [49]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
# document_num = 4310
bow_corpus[document_num]

[(30, 1),
 (123, 1),
 (124, 1),
 (177, 1),
 (199, 1),
 (335, 1),
 (434, 1),
 (482, 1),
 (485, 1),
 (530, 1),
 (539, 2),
 (583, 1),
 (586, 1),
 (628, 1),
 (746, 1),
 (778, 1),
 (984, 1),
 (1086, 1),
 (1303, 1),
 (1633, 2),
 (2193, 1),
 (2296, 2)]

In [50]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 30 ("disk") appears 1 time.
Word 123 ("take") appears 1 time.
Word 124 ("use") appears 1 time.
Word 177 ("file") appears 1 time.
Word 199 ("fact") appears 1 time.
Word 335 ("pretti") appears 1 time.
Word 434 ("sourc") appears 1 time.
Word 482 ("reason") appears 1 time.
Word 485 ("sound") appears 1 time.
Word 530 ("generat") appears 1 time.
Word 539 ("program") appears 2 time.
Word 583 ("conserv") appears 1 time.
Word 586 ("coupl") appears 1 time.
Word 628 ("interest") appears 1 time.
Word 746 ("matter") appears 1 time.
Word 778 ("hardwar") appears 1 time.
Word 984 ("long") appears 1 time.
Word 1086 ("minut") appears 1 time.
Word 1303 ("special") appears 1 time.
Word 1633 ("bit") appears 2 time.
Word 2193 ("purpos") appears 1 time.
Word 2296 ("random") appears 2 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [51]:
'''
Create tf-idf model object on 
'''
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

In [52]:
'''
Apply transformation to the entire corpus
'''
corpus_tfidf = tfidf[bow_corpus]

In [54]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.1875264581347021),
 (1, 0.19048818244598065),
 (2, 0.3200391084578768),
 (3, 0.13734351513213772),
 (4, 0.4474660769851411),
 (5, 0.1942033579423499),
 (6, 0.17810397981137643),
 (7, 0.3120247629296378),
 (8, 0.1888992200040353),
 (9, 0.17867218585267072),
 (10, 0.2065318154009039),
 (11, 0.13825450801956224),
 (12, 0.184086472847616),
 (13, 0.18071128387952717),
 (14, 0.18283363306153416),
 (15, 0.2065318154009039),
 (16, 0.1682262661361903),
 (17, 0.23995085429482482),
 (18, 0.23341628184742005),
 (19, 0.16695673871225614)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999`
    
At a high level:

* Go through each document, and randomly assign each word in the document to one of the K topics. Notice that this random assignment already gives you both topic representations of all the documents and word distributions of all the topics (albeit not very good ones).
* So to improve on them, for each document d:
    - Go through each word w in d:
        - And for each topic t, compute two things: 
            - 1) A - the proportion of words in document `d` that are currently assigned to topic `t`, and 
            - 2) B - the proportion of assignments to topic `t` over all documents that come from this word `w`. 
        - Reassign `w` a new topic, where we choose topic `t` with probability `A * B` (according to our generative model, this is essentially the probability that topic `t` generated word `w`, so it makes sense that we resample the current word’s topic with this probability). 
        
        
* After repeating the previous step a large number of times, you’ll eventually reach a roughly steady state where your assignments are pretty good. So use these assignments to estimate the topic mixtures of each document:
    - by counting the proportion of words assigned to each topic within that document and 
    - the words associated to each topic (by counting the proportion of words assigned to each topic overall|.    

In [57]:
# LDA mono-core
# lda_model = gensim.models.LdaModel(bow_corpus, 
#                                    num_topics = 10, 
#                                    id2word = dictionary,                                    
#                                    passes = 1)

# LDA multicore 
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=10, 
                                       id2word = dictionary, 
                                       passes = 50, 
                                       workers=2)

In [56]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.023*"space" + 0.009*"nasa" + 0.008*"research" + 0.008*"launch" + 0.008*"orbit" + 0.007*"univers" + 0.007*"center" + 0.007*"develop" + 0.007*"program" + 0.007*"data"


Topic: 1 Word: 0.014*"encrypt" + 0.012*"secur" + 0.011*"chip" + 0.010*"mail" + 0.010*"public" + 0.010*"inform" + 0.008*"govern" + 0.008*"key" + 0.008*"messag" + 0.008*"clipper"


Topic: 2 Word: 0.017*"window" + 0.014*"file" + 0.010*"drive" + 0.010*"program" + 0.009*"card" + 0.007*"version" + 0.007*"softwar" + 0.007*"disk" + 0.006*"imag" + 0.006*"avail"


Topic: 3 Word: 0.012*"israel" + 0.011*"kill" + 0.010*"jew" + 0.009*"govern" + 0.009*"isra" + 0.007*"attack" + 0.007*"arab" + 0.007*"countri" + 0.007*"forc" + 0.007*"state"


Topic: 4 Word: 0.013*"state" + 0.012*"price" + 0.011*"sell" + 0.009*"sale" + 0.008*"money" + 0.008*"presid" + 0.008*"govern" + 0.007*"control" + 0.007*"offer" + 0.007*"cost"


Topic: 5 Word: 0.028*"game" + 0.021*"team" + 0.017*"play" + 0.013*"player" + 0.010*"season" + 0.009*"hockey" 

### Classification of the topics ###

Using the words in each topic and their corresponding weights, we have the following categories:

* 0: Computer Software
* 1: Space
* 2: Government
* 3: Energy
* 4: Sports
* 5: Applications
* 6: Religion
* 7: Computer Hardware
* 8: Automotives
* 9: Politics

## Step 4.2 Running LDA using TF-IDF ##

In [58]:
'''
Define lda model using tfidf corpus
'''
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             num_topics=10, 
                                             id2word=dictionary, 
                                             passes=50, 
                                             workers=4)

In [59]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.025*"israel" + 0.020*"isra" + 0.016*"arab" + 0.009*"palestinian" + 0.007*"jew" + 0.007*"adam" + 0.006*"lebanon" + 0.006*"address" + 0.006*"david" + 0.006*"april"


Topic: 1 Word: 0.010*"captain" + 0.010*"midi" + 0.010*"saturn" + 0.008*"blow" + 0.007*"queen" + 0.007*"bronx" + 0.007*"manhattan" + 0.006*"beauchain" + 0.006*"bobb" + 0.006*"sink"


Topic: 2 Word: 0.024*"bike" + 0.009*"ride" + 0.009*"motorcycl" + 0.008*"mile" + 0.008*"car" + 0.007*"honda" + 0.007*"rear" + 0.007*"rid" + 0.006*"road" + 0.006*"drive"


Topic: 3 Word: 0.019*"window" + 0.017*"file" + 0.010*"program" + 0.007*"server" + 0.007*"version" + 0.007*"graphic" + 0.006*"advanc" + 0.006*"font" + 0.006*"code" + 0.005*"mail"


Topic: 4 Word: 0.010*"card" + 0.009*"drive" + 0.006*"disk" + 0.006*"monitor" + 0.005*"video" + 0.005*"price" + 0.005*"driver" + 0.005*"scsi" + 0.005*"control" + 0.005*"sale"


Topic: 5 Word: 0.011*"detector" + 0.010*"colormap" + 0.009*"radar" + 0.008*"doug" + 0.007*"upenn" + 0.007*"tige

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, we have the following categories:

* 0: Unclear
* 1: Computer Software
* 2: Space
* 3: Religion
* 4: Middle East
* 5: Unclear
* 6: Unclear
* 7: Politics
* 8: Automotives
* 9: Sports

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [69]:
'''
Text of sample document 5288
'''
print(documents[document_num])

Have anybody succeded in converting a atari monomchrome monitor into a
mono VGA monitor. If so please let me know exactly how you did and what
graphics card you used.

	/Thanx


In [70]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.8239408731460571	 
Topic: 0.020*"drive" + 0.015*"card" + 0.011*"disk" + 0.010*"window" + 0.009*"control" + 0.009*"scsi" + 0.009*"driver" + 0.007*"memori" + 0.007*"hard" + 0.007*"price"

Score: 0.10332677513360977	 
Topic: 0.022*"file" + 0.013*"program" + 0.012*"window" + 0.008*"imag" + 0.008*"avail" + 0.008*"encrypt" + 0.008*"mail" + 0.007*"inform" + 0.007*"server" + 0.007*"version"


### It has the highest probability (`82.4%`) to be  part of the topic that we assigned as Computer Hardware, which is the accurate classification. ###

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [71]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6669110655784607	 
Topic: 0.010*"card" + 0.009*"drive" + 0.006*"disk" + 0.006*"monitor" + 0.005*"video" + 0.005*"price" + 0.005*"driver" + 0.005*"scsi" + 0.005*"control" + 0.005*"sale"

Score: 0.2603572607040405	 
Topic: 0.019*"window" + 0.017*"file" + 0.010*"program" + 0.007*"server" + 0.007*"version" + 0.007*"graphic" + 0.006*"advanc" + 0.006*"font" + 0.006*"code" + 0.005*"mail"


### It has the highest probability (`66.7%`) to be  part of the topic that we assigned as Computer Hardware. ###

## Step 5.2: Performance evaluation of model as a whole ##

We use the perplexity(logarithm) for our evaluation. The perplexity is connected to the log likelihood that the model is able to **generate the documents, given the distribution of topics for those documents**. The lower the perplexity, the better the model as it signifies that the model can regenerate the text quite well.

In [73]:
'''
Calcualte log perplexity of the model to check if the model can generate the text.
'''
print("Log perplexity of the model is", lda_model.log_perplexity(bow_corpus))

Log perplexity of the model is -7.371806882346939


## Step 6: Testing model on unseen document ##

In [74]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6090230941772461	 Topic: 0.029*"game" + 0.022*"team" + 0.018*"play" + 0.014*"player" + 0.011*"season"
Score: 0.230947345495224	 Topic: 0.025*"space" + 0.010*"nasa" + 0.009*"launch" + 0.009*"orbit" + 0.007*"program"
Score: 0.02000638097524643	 Topic: 0.008*"valu" + 0.008*"differ" + 0.008*"caus" + 0.008*"effect" + 0.008*"case"
Score: 0.020005231723189354	 Topic: 0.020*"drive" + 0.015*"card" + 0.011*"disk" + 0.010*"window" + 0.009*"control"
Score: 0.02000388875603676	 Topic: 0.010*"univers" + 0.010*"inform" + 0.010*"state" + 0.008*"articl" + 0.008*"book"
Score: 0.020003557205200195	 Topic: 0.011*"govern" + 0.008*"state" + 0.007*"presid" + 0.005*"public" + 0.005*"mean"
Score: 0.020003268495202065	 Topic: 0.007*"wire" + 0.006*"bike" + 0.006*"littl" + 0.005*"tell" + 0.005*"leav"
Score: 0.02000322751700878	 Topic: 0.022*"file" + 0.013*"program" + 0.012*"window" + 0.008*"imag" + 0.008*"avail"
Score: 0.020002182573080063	 Topic: 0.022*"armenian" + 0.011*"israel" + 0.011*"kill" + 0.010*

The model correctly classifies the unseen document with 61.6% probability to the Sports category.